In [ ]:
import os, sys
import gspread, httplib2
from oauth2client.client import OAuth2WebServerFlow
from oauth2client import file, tools
from google_ids import *

filename = "oauth.dat"

class common_date_boundaries():
    """Create a selection of dates that can be used as arguments to Analytics or Search Console"""
    def __init__(self):
        from datetime import date, time, datetime, timedelta
        shift = 2
        today = date.today()
        yesterday = today - timedelta(days=1)
        day = today - timedelta(days=shift)
        week = day.isocalendar()[1]
        year = day.year
        first_day_this_month = day.replace(day=1)
        first_week_of_year = date(year,1,1)
        if(first_week_of_year.weekday()>3):
            first_week_of_year = first_week_of_year+timedelta(7-first_week_of_year.weekday())
        else:
            first_week_of_year = first_week_of_year - timedelta(first_week_of_year.weekday())
        prior_weeks_of_year = timedelta(days = (week-1)*7)
        week_start = first_week_of_year + prior_weeks_of_year
        week_end = first_week_of_year + prior_weeks_of_year + timedelta(days=6)
        week_start = week_start + timedelta(days=-1)
        week_end = week_end + timedelta(days=-1)
        month_end = first_day_this_month - timedelta(days=1)
        month_start = month_end.replace(day=1)
        month_days = (month_end - month_start).days+1
        start_30_days = day - timedelta(days=30)
        end_30_days = day
        start_90_days = day - timedelta(days=90) - timedelta(days=shift-1)
        end_90_days = day
        props = ['month_days','year','week',
                 'first_week_of_year',
                 'today','day',
                 'first_day_this_month',
                 'month_start','month_end',
                 'week_start','week_end',
                 'start_30_days','end_30_days',
                 'start_90_days','end_90_days'
                ]
        for prop in props:
            command = "self.%s = str(%s)" % (prop, prop)
            exec(command)


class Unbuffered(object):
    def __init__(self, stream):
        self.stream = stream
    def write(self, data):
        self.stream.write(data)
        self.stream.flush()
    def __getattr__(self, attr):
        return getattr(self.stream, attr)


class MyCreds (object): 
    def __init__ (self, access_token=None): 
        self.access_token = access_token 


def get_email():
    """May end up moving this into GoodSheet"""
    service = create_google_service(filename=filename, api_name="oauth2", version="v2")
    user_document = service.userinfo().get().execute()
    email = user_document['email']
    return email


def create_google_service(filename, api_name, version):
    """This lets you create instances of Google Services."""
    from apiclient.discovery import build
    path = os.path.dirname(os.path.realpath('__file__'))
    path_filename = os.path.join(path, filename)
    storage = file.Storage(path_filename)
    credentials = storage.get()
    http = credentials.authorize(http = httplib2.Http())
    service = build(api_name, version, http)
    return service


def oauth():
    import argparse, json
    scopes = ["https://www.googleapis.com/auth/analytics.readonly", 
              "https://www.googleapis.com/auth/webmasters.readonly",
              "https://www.googleapis.com/auth/yt-analytics.readonly",
              "https://www.googleapis.com/auth/youtube.readonly",
              "https://spreadsheets.google.com/feeds/",
              "https://www.googleapis.com/auth/gmail.modify",
              "https://www.googleapis.com/auth/userinfo.email"]
    
    path = os.path.dirname(os.path.realpath('__file__'))
    path_filename = os.path.join(path, filename)
    flow = OAuth2WebServerFlow(client_id, client_secret, scopes, 
                               redirect_uri='urn:ietf:wg:oauth:2.0:oob',
                               response_type='code',
                               prompt='consent',
                               access_type='offline')
    authorize_url = flow.step1_get_authorize_url()
    storage = file.Storage(path_filename)
    credentials = storage.get()
    argparser = argparse.ArgumentParser(add_help=False)
    parents = [argparser]
    parent_parsers = [tools.argparser]
    parent_parsers.extend(parents)
    parser = argparse.ArgumentParser(
        description="__doc__",
        formatter_class=argparse.RawDescriptionHelpFormatter,
        parents=parent_parsers)
    flags = parser.parse_args(['--noauth_local_webserver'])
    try:
        http = credentials.authorize(http = httplib2.Http())
    except:
        pass
    if credentials is None or credentials.invalid:
        credentials = tools.run_flow(flow, storage, flags)
    else:
        credentials.refresh(http)
    with open(path_filename) as json_file:
        jdata = json.load(json_file)
    token = jdata['access_token']
    creds = MyCreds(access_token=token)
    connect = gspread.authorize(creds)
    return connect

oauth()

# Force IPython Notebook to not buffer output
sys.stdout = Unbuffered(sys.stdout)